In [36]:
from dotenv import load_dotenv
import os
load_dotenv()
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
import json
from langchain_core.messages import ToolMessage, HumanMessage

# Define state type
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize tools
tool = TavilySearchResults(max_results=2)
tools = [tool]

# Create LLM with tools
llm = ChatOllama(model="qwen2.5:3b")
llm_with_tools = llm.bind_tools(tools)

# Tool node implementation
class BasicToolNode:
    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}
    
    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No messages found in the inputs")
        
        outputs = []
        
        # Parse tool calls from the content if they're in the format shown in the result
        content = message.content
        if "<tool_call>" in content:
            # Extract the JSON from between <tool_call> tags
            import re
            tool_call_matches = re.findall(r'<tool_call>\s*(.*?)\s*</tool_call>', content, re.DOTALL)
            
            for tool_call_json in tool_call_matches:
                try:
                    tool_call = json.loads(tool_call_json)
                    tool_name = tool_call.get("name")
                    tool_args = tool_call.get("arguments", {})
                    
                    if tool_name in self.tools_by_name:
                        tool_result = self.tools_by_name[tool_name].invoke(tool_args)
                        outputs.append(
                            ToolMessage(
                                content=json.dumps(tool_result),
                                name=tool_name,
                                tool_call_id=tool_name  # Using tool_name as ID since we don't have one
                            )
                        )
                except json.JSONDecodeError:
                    continue
        
        # Also check for standard tool_calls attribute if available
        elif hasattr(message, "tool_calls") and message.tool_calls:
            for tool_call in message.tool_calls:
                tool_result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])
                outputs.append(
                    ToolMessage(
                        content=json.dumps(tool_result),
                        name=tool_call["name"],
                        tool_call_id=tool_call["id"]
                    )
                )
                
        return {"messages": outputs}

# Initialize the workflow
workflow = StateGraph(State)

# Create nodes
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

tool_node = BasicToolNode(tools=tools)

# Add nodes to workflow
workflow.add_node("chatbot", chatbot)
workflow.add_node("tools", tool_node)

# Define edge routing logic
def route_tools(state: State):
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    
    # Check if the message has tool_calls attribute
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    
    # Also check for tool calls in content (format: <tool_call>{"name": "...", "arguments": {...}}</tool_call>)
    if hasattr(ai_message, "content") and "<tool_call>" in ai_message.content:
        return "tools"
    
    return END

# Add edges
workflow.add_conditional_edges(
    "chatbot",
    route_tools,
    {
        "tools": "tools",
        END: END
    }
)
workflow.add_edge("tools", "chatbot")
workflow.add_edge(START, "chatbot")

# Compile the graph
graph = workflow.compile()

# Example usage
result = graph.invoke({"messages": [HumanMessage(content="What is the current political news on India and Bangladesh")]})
print(result)

{'messages': [HumanMessage(content='What is the current political news on India and Bangladesh', additional_kwargs={}, response_metadata={}, id='5cf0319b-c7f0-4e74-a497-ebe1365ab1c8'), AIMessage(content='<tool_call>\n{"name": "tavily_search_results_json", "arguments": {"query": "India politics Bangladesh politics"}}\n</tool_call>', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-04-06T19:33:59.0506721Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3389521200, 'load_duration': 33161400, 'prompt_eval_count': 197, 'prompt_eval_duration': 445000000, 'eval_count': 27, 'eval_duration': 2895000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-15aa72a3-5e34-4e84-b4cf-3ff9d092f601-0', usage_metadata={'input_tokens': 197, 'output_tokens': 27, 'total_tokens': 224}), ToolMessage(content='[{"title": "Bangladesh\\u2013India relations - Wikipedia", "url": "https://en.wikipedia.org/wiki/Bangladesh%E2%80%93India_

In [41]:
print(result["messages"][-1].content)

Based on the search results provided by Tavily:

One of the articles is about Bangladesh-India relations, mentioning anti-Indian sentiments among Bangladesh citizens due to issues like the Indian government's interference in internal politics of Bangladesh, killings of Bangladeshi people by Indian BSF, Citizenship Amendment Act, and Hindutva in India. Despite these challenges, there have been notable developments such as resolving land and maritime boundaries disputes.

Another article talks about recent political shifts in Bangladesh where the resurgence of the Bangladesh Nationalist Party (BNP) and Jamaat-e-Islami (JI) has caused concerns for India. 

It seems like both countries share many cultural ties due to shared Bengali-speaking populations, especially in West Bengal and Tripura states of India.

While some disputes remain unresolved, there have been cooperative efforts between them as common members of SAARC, BIMSTEC, IORA and the Commonwealth.
